In [87]:
# Parses FASTA file using BioPython Library

from Bio import SeqIO

for seq_record in SeqIO.parse("6ezq.fasta.txt", "fasta"):
    print((seq_record.id))
    print((repr(seq_record.seq)))
    print((len(seq_record)))

6EZQ:A|PDBID|CHAIN|SEQUENCE
Seq('DAHKSEVAHRFKDLGEENFKALVLIAFAQYLQQCPFEDHVKLVNEVTEFAKTCV...LGL', SingleLetterAlphabet())
585


In [88]:
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq

my_seq = seq_record.seq
# for index, letter in enumerate(my_seq):
#     print("%i %s" % (index, letter))

In [89]:
from Bio.Seq import Seq
from Bio import Alphabet
from Bio.Alphabet import Reduced
my_protein = seq_record.seq
# Used BioPython hp conversion function from Library. Tested with the
# Amino Acid Properties to be the same classification

new_protein = Seq('', Alphabet.Reduced.HPModel())

for aa in my_protein:
    new_protein += Alphabet.Reduced.hp_model_tab[aa]


new_protein


Seq('PPPPPPHPPPHPPHPPPPHPPHHHHPHPPHHPPHPHPPPHPHHPPHPPHPPPHH...HPH', HPModel())

In [83]:
#new_protein = 'HPPPPPPHH'

In [71]:
# 0:F 1:L 2:R
# generate random integer values
from random import seed
from random import randint
from random import choice
# seed random number generator


def rnd_solution_generator(num_sol):

    seed(1)
    all_solutions = []
    for j in range(num_sol):

        new_pop = []
        counter = 0
        for i in new_protein:
            if (counter > 2):
                if (new_pop[counter - 1] == new_pop[counter - 2]):
                    rnd_Num = choice([
                        i for i in range(0, 2)
                        if i not in [new_pop[counter - 1]]
                    ])
                else:
                    rnd_Num = randint(0, 2)

            else:
                rnd_Num = randint(0, 2)
            new_pop.append(rnd_Num)
            counter += 1
        all_solutions.append(new_pop)
        len(new_pop)
    return all_solutions


# all_sol = rnd_solution_generator(2)
# len(all_sol)

In [76]:
# ##CREATES A BLANK MATRIX WITH EMPTY CHARACTERS CURRENT_GRID
# #-Fill the grid with empty characters-#
# # For each row of the grid
# current_element_idx = 0
# current_grid = []
    
# for row in range(len(new_protein) * 2 - 1):

#     # Create a new empty list for that row
#     current_grid.append([])

#     # For each column of the grid
#     for col in range(len(new_protein) * 2 - 1):

#         # Add an empty character at the given row/column
#         current_grid[row].append(EMPTY)

# # Start the first protein element in the middle of the grid
# current_row = current_col = int((len(new_protein) * 2 - 1) / 2)

In [77]:
import numpy as np
import pandas as pd
empty_grid = np.empty((len(new_protein)*2, len(new_protein)*2))
empty_grid.shape

(20, 20)

In [72]:
all_sol = rnd_solution_generator(100)
len(all_sol)

100

In [73]:
all_sol[0]

[0, 2, 0, 1, 0, 1, 1, 0, 2, 1]

In [ ]:
# 0:F 1:L 2:R
def fitness_score(solution, new_protein):
    
    #orientation -1:left 0:neutral 1:right 2:up -2:down
    orientation = 0
    
    for i in solution:
        if(i == 0):
            
    
    
    

In [49]:

#CURRENTLY WORKING ON EDITING THIS FUNCTION TO MAKE IT WORK BY SCORING JUST 1 MATRIX AT A TIME
#It works recursively

def fold(protein, current_element_idx, current_grid,
         current_row, current_col, direction, orientation, solution):

    # Determine the new current row and column based on the current
    # direction
    if(orientation == )
    if direction == 'R':
        current_col += 1
    elif direction == 'D':
        current_row += 1
    elif direction == 'L':
        current_col -= 1
    elif direction == 'U':
        current_row -= 1

    # If we are able to place an element at the current row and column
    if current_grid[current_row][current_col] == EMPTY:

        #-Make a copy of the current grid before we change it-#
        new_grid = []
        for row in range(len(protein) * 2 - 1):
            new_grid.append([])
            for col in range(len(protein) * 2 - 1):
                new_grid[row].append(current_grid[row][col])

        # Place the protein in the new grid
        new_grid[current_row][current_col] = protein[current_element_idx]

        # Check for H-H bonds in the current fold
        if protein[current_element_idx] == 'H':

            # Check to the left
            if current_col > 0 and new_grid[current_row][current_col -
                                                         1] == 'H':
                current_num_H_bonds += 1

            # Check above
            if current_row > 0 and new_grid[current_row -
                                            1][current_col] == 'H':
                current_num_H_bonds += 1

            # Check to the right
            if current_col < len(new_grid[current_row]) - 1 and new_grid[
                    current_row][current_col + 1] == 'H':
                current_num_H_bonds += 1

            # Check below
            if current_row < len(new_grid) - 1 and new_grid[
                    current_row + 1][current_col] == 'H':
                current_num_H_bonds += 1

        # Move on to the next element index
        current_element_idx += 1

        # If not end of string, choose each direction and recur
        if current_element_idx != len(protein):
            for direction in solution:
                max_num_H_bonds = fold(protein,
                                       current_element_idx, new_grid,
                                       current_row, current_col, direction,
                                       current_num_H_bonds)

        else:
            # If end of string, check if the current fold has more
            # H-H bonds than the max we found before.
            # if true update the max.  If we have the same number of
            # H-H bonds, append the current grid to the list of best
            # grids
            if current_num_H_bonds > max_num_H_bonds:
                max_num_H_bonds = current_num_H_bonds
                del best_grids[:]
                best_grids.append(new_grid)
            elif current_num_H_bonds == max_num_H_bonds:
                best_grids.append(new_grid)

    # Return the count of the maximum number of H-H bonds
    return max_num_H_bonds


1169

In [296]:
# import sys

# EMPTY = ' '  # This represents an empty character in the grid
# best_grids = []  # This is a list of the grids that contain the best foldings

# #---Main function definition---#


# def main():

#     #--Local variable initialization--#
#     current_element_idx = 0  # Index of the current element in protein
#     protein = 'HPPPPPPHH'  # Protein we are folding
#     #protein = "HHPPHHHPHHPH" # Protein we are folding
#     current_grid = []  # Grid in which we are currently folding
#     current_num_H_bonds = 0
#     max_num_H_bonds = 0  # Counters
#     print("Hello protein folders, beginning program")  # Print a greeting

#     #-Fill the grid with empty characters-#
#     # For each row of the grid
#     for row in range(len(protein) * 2 - 1):

#         # Create a new empty list for that row
#         current_grid.append([])

#         # For each column of the grid
#         for col in range(len(protein) * 2 - 1):

#             # Add an empty character at the given row/column
#             current_grid[row].append(EMPTY)

#     # Start the first protein element in the middle of the grid
#     current_row = current_col = int((len(protein) * 2 - 1) / 2)

#     # Recursively find the best foldings, filling best_grid
#     print(("Best number of H-H bonds",
#            fold(protein, max_num_H_bonds, current_element_idx, current_grid,
#                 current_row, current_col, ' ', 2, current_num_H_bonds)))

#     #-Print the best grids-#
#     # For each of the best grids
#     for grid in range(len(best_grids)):

#         # Print the index of the grid
#         print("Grid ", grid, ":")

#         # For each row in the grid
#         for row in range(len(protein) * 2 - 1):

#             # For each column in the grid
#             for col in range(len(protein) * 2 - 1):

#                 # Print the character at the given row and
#                 # column
#                 print(best_grids[grid][row][col], end=' ')

#             # Print a line in between the rows
#             print('')


# #---Fold function definition---#


# def fold(protein, max_num_H_bonds, current_element_idx, current_grid,
#          current_row, current_col, direction, orientation, current_num_H_bonds):

#     # Determine the new current row and column based on the current
#     # direction
#      #orientation -1:left 1:right 2:up -2:down  
#     rel_direction = ''
#     if (orientation == -1):
#         if(direction == 'L'):

#             rel_direction = 'F'
#             orientation = -1

#         elif(direction == 'D'):

#             rel_direction = 'L'
#             orientation = -2
            
#         elif(direction == 'U'):

#             rel_direction = 'R'
#             orientation = 2
            
#     elif (orientation == 1):
#         if(direction == 'R'):
            
#             rel_direction = 'F'
#             orientation = 1
            
#         elif(direction == 'U'):
            
#             rel_direction = 'L'
#             orientation = 2
            
#         elif(direction == 'D'):
            
#             rel_direction = 'R'
#             orientation = -2
    
#     elif (orientation == 2):
#         if(direction == 'U'):

#             rel_direction = 'F'
#             orientation = 2
            
#     elif (orientation == -2):
#         if(direction == 'D'):

#             rel_direction = 'F'
#             orientation = -2
            
#         elif(direction == 'R'):

#             rel_direction = 'L'
#             orientation = 1
            
#         elif(direction == 'L'):

#             rel_direction = 'R'
#             orientation = -1
    
    
    
    
    
#     if direction == 'R':
#         current_col += 1
#     elif direction == 'D':
#         current_row += 1
#     elif direction == 'L':
#         current_col -= 1
#     elif direction == 'U':
#         current_row -= 1

#     # If we are able to place an element at the current row and column
#     if current_grid[current_row][current_col] == EMPTY:

#         #-Make a copy of the current grid before we change it-#
#         new_grid = []
#         for row in range(len(protein) * 2 - 1):
#             new_grid.append([])
#             for col in range(len(protein) * 2 - 1):
#                 new_grid[row].append(current_grid[row][col])

#         # Place the protein in the new grid
#         new_grid[current_row][current_col] = protein[current_element_idx]

#         # Check for H-H bonds in the current fold
#         if protein[current_element_idx] == 'H':

#             # Check to the left
#             if current_col > 0 and new_grid[current_row][current_col -
#                                                          1] == 'H':
#                 current_num_H_bonds += 1

#             # Check above
#             if current_row > 0 and new_grid[current_row -
#                                             1][current_col] == 'H':
#                 current_num_H_bonds += 1

#             # Check to the right
#             if current_col < len(new_grid[current_row]) - 1 and new_grid[
#                     current_row][current_col + 1] == 'H':
#                 current_num_H_bonds += 1

#             # Check below
#             if current_row < len(new_grid) - 1 and new_grid[
#                     current_row + 1][current_col] == 'H':
#                 current_num_H_bonds += 1

#         # Move on to the next element index
#         current_element_idx += 1

#         # If not end of string, choose each direction and recur
#         if current_element_idx != len(protein):
#             for direction in ['R', 'D', 'L', 'U']:
#                 max_num_H_bonds = fold(protein, max_num_H_bonds,
#                                        current_element_idx, new_grid,
#                                        current_row, current_col, direction,
#                                        orientation, current_num_H_bonds)

#         else:
#             # If end of string, check if the current fold has more
#             # H-H bonds than the max we found before.
#             # if true update the max.  If we have the same number of
#             # H-H bonds, append the current grid to the list of best
#             # grids
#             if current_num_H_bonds > max_num_H_bonds:
#                 max_num_H_bonds = current_num_H_bonds
#                 del best_grids[:]
#                 best_grids.append(new_grid)
#             elif current_num_H_bonds == max_num_H_bonds:
#                 best_grids.append(new_grid)

#     # Return the count of the maximum number of H-H bonds
#     return max_num_H_bonds


# ##---Call main function-----##
# main()

In [318]:
import sys
import numpy as np
import copy
EMPTY = ' '  # This represents an empty character in the grid
best_grids = []  # This is a list of the grids that contain the best foldings
direction_matrix = []
score_matrix = []
#---Main function definition---#


def main():

    #--Local variable initialization--#
    current_element_idx = 0  # Index of the current element in protein
    protein = 'HPPPPPPHH'  # Protein we are folding
    #protein = "HHPPHHHPHHPH" # Protein we are folding
    current_grid = []  # Grid in which we are currently folding
    current_num_H_bonds = 0
    max_num_H_bonds = 0  # Counters
    print("Hello protein folders, beginning program")  # Print a greeting

    #-Fill the grid with empty characters-#
    # For each row of the grid
    for row in range(len(protein) * 2 - 1):

        # Create a new empty list for that row
        current_grid.append([])

        # For each column of the grid
        for col in range(len(protein) * 2 - 1):

            # Add an empty character at the given row/column
            current_grid[row].append(EMPTY)

    # Start the first protein element in the middle of the grid
    current_row = current_col = int((len(protein) * 2 - 1) / 2)
    
    new_direction_grid = []
    orientation = 2
    # Recursively find the best foldings, filling best_grid
    print(("Best number of H-H bonds",
           fold(protein, max_num_H_bonds, current_element_idx, current_grid,
                current_row, current_col, ' ', current_num_H_bonds, new_direction_grid, orientation)))
    
    print(direction_matrix[0])
#     for i in direction_matrix:
#         print(i)
#     print(len(direction_matrix))
    #-Print the best grids-#
    # For each of the best grids
    for grid in range(len(best_grids)):

        # Print the index of the grid
        print("Grid ", grid, ":")

        # For each row in the grid
        for row in range(len(protein) * 2 - 1):

            # For each column in the grid
            for col in range(len(protein) * 2 - 1):

                # Print the character at the given row and
                # column
                print(best_grids[grid][row][col], end=' ')

            # Print a line in between the rows
            print('')


#---Fold function definition---#


def fold(protein, max_num_H_bonds, current_element_idx, current_grid,
         current_row, current_col, direction, current_num_H_bonds, new_direction_grid, orientation):
    #orientation -1:left 1:right 2:up -2:down  
    # Determine the new current row and column based on the current
    # direction
    rel_direction = ''
    if (orientation == -1):
        if(direction == 'L'):

            rel_direction = 'F'
            orientation = -1

        elif(direction == 'D'):

            rel_direction = 'L'
            orientation = -2
            
        elif(direction == 'U'):

            rel_direction = 'R'
            orientation = 2
            
            
    elif (orientation == 1):
        if(direction == 'R'):
            
            rel_direction = 'F'
            orientation = 1
            
        elif(direction == 'U'):
            
            rel_direction = 'L'
            orientation = 2
            
        elif(direction == 'D'):
            
            rel_direction = 'R'
            orientation = -2
    
    
    elif (orientation == 2):
        if(direction == 'U'):

            rel_direction = 'F'
            orientation = 2
            
        elif(direction == 'L'):
            rel_direction = 'L'
            orientation = -1
        
        elif(direction == 'R'):
            rel_direction = 'R'
            orientation = 1      
            
    elif (orientation == -2):
        if(direction == 'D'):

            rel_direction = 'F'
            orientation = -2
            
        elif(direction == 'R'):

            rel_direction = 'L'
            orientation = 1
            
        elif(direction == 'L'):

            rel_direction = 'R'
            orientation = -1
    
    
    
    
    
    if direction == 'R':
        current_col += 1
    elif direction == 'D':
        current_row += 1
    elif direction == 'L':
        current_col -= 1
    elif direction == 'U':
        current_row -= 1

    # If we are able to place an element at the current row and column
    if current_grid[current_row][current_col] == EMPTY:
#         np.append(direction_matrix, direction)
#         direction_matrix.append()
        #-Make a copy of the current grid before we change it-#
        new_grid = []
        new_direction_grid_temp = ''
        new_direction_grid_temp = copy.deepcopy(new_direction_grid)
        new_direction_grid_temp += rel_direction
        for row in range(len(protein) * 2 - 1):
            new_grid.append([])
            for col in range(len(protein) * 2 - 1):
                new_grid[row].append(current_grid[row][col])

        # Place the protein in the new grid
        new_grid[current_row][current_col] = protein[current_element_idx]

        # Check for H-H bonds in the current fold
        if protein[current_element_idx] == 'H':

            # Check to the left
            if current_col > 0 and new_grid[current_row][current_col -
                                                         1] == 'H':
                current_num_H_bonds += 1

            # Check above
            if current_row > 0 and new_grid[current_row -
                                            1][current_col] == 'H':
                current_num_H_bonds += 1

            # Check to the right
            if current_col < len(new_grid[current_row]) - 1 and new_grid[
                    current_row][current_col + 1] == 'H':
                current_num_H_bonds += 1

            # Check below
            if current_row < len(new_grid) - 1 and new_grid[
                    current_row + 1][current_col] == 'H':
                current_num_H_bonds += 1

        # Move on to the next element index
        current_element_idx += 1

        # If not end of string, choose each direction and recur
        if current_element_idx != len(protein):
            for direction in ['R', 'D', 'L', 'U']:
                max_num_H_bonds = fold(protein, max_num_H_bonds,
                                       current_element_idx, new_grid,
                                       current_row, current_col, direction,
                                       current_num_H_bonds,  new_direction_grid_temp, orientation)

        else:
            # If end of string, check if the current fold has more
            # H-H bonds than the max we found before.
            # if true update the max.  If we have the same number of
            # H-H bonds, append the current grid to the list of best
            # grids
            if current_num_H_bonds > max_num_H_bonds:
                max_num_H_bonds = current_num_H_bonds
#               del best_grids[:]
#               del direction_matrix[:]
                best_grids.append(new_grid)
                score_matrix.append(current_num_H_bonds)
                direction_matrix.append(new_direction_grid_temp)
            elif current_num_H_bonds == max_num_H_bonds:
                best_grids.append(new_grid)
                score_matrix.append(current_num_H_bonds)
                direction_matrix.append(new_direction_grid_temp)

    # Return the count of the maximum number of H-H bonds
    return max_num_H_bonds

#[' ', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'U']
##---Call main function-----##
main()

Hello protein folders, beginning program
('Best number of H-H bonds', 2)
['R', 'F', 'F', 'F', 'F', 'F', 'F', 'F']
Grid  0 :
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                H P P P P P P H H 
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                                  
Grid  1 :
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                          

                                  
                                  
                                  
                H P P P P P       
                      H H P       
                                  
                                  
                                  
                                  
                                  
                                  
                                  
Grid  17 :
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                          P H H   
                H P P P P P       
                                  
                                  
                                  
                                  
                                  
                                  
                                  
         

                                  
                                  
                H P P P P H H     
                        P P       
                                  
                                  
                                  
                                  
                                  
                                  
                                  
Grid  32 :
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                          H       
                H P P P P H       
                        P P       
                                  
                                  
                                  
                                  
                                  
                                  
                                  
Grid  33 

                                  
                                  
Grid  46 :
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                H P P P P         
                  H H P P         
                                  
                                  
                                  
                                  
                                  
                                  
                                  
Grid  47 :
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                        P P H H   
                H P P P P        

                                  
                                  
Grid  61 :
                                  
                                  
                                  
                                  
                                  
                                  
                        P H       
                        P H       
                H P P P P         
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                                  
Grid  62 :
                                  
                                  
                                  
                                  
                                  
                          H       
                        P H       
                        P         
                H P P P P        

...

                                  
Grid  225 :
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                                  
              P H                 
            P P H H               
            P P P                 
                                  
                                  
                                  
                                  
                                  
                                  
Grid  226 :
                                  
                                  
                                  
                                  
                                  
                                  
                                  
            P P H H               
            P P H                 
            P P                

              P P                 
            P P H H               
            P P H                 
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                                  
Grid  242 :
                                  
                                  
                                  
                                  
                                  
                                  
            P P P                 
            P   H H               
            P P H                 
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                                  
Grid  24

                                  
                                  
                                  
                                  
                                  
                                  
                                  
Grid  258 :
                                  
                                  
                                  
                                  
                                  
              P P                 
              P P                 
              P H H               
              P H                 
                                  
                                  
                                  
                                  
                                  
                                  
                                  
                                  
Grid  259 :
                                  
                                  
                                  
                               

                                  
                                  
Grid  272 :
                                  
                                  
                                  
                                  
                                  
                                  
                                  
              P P                 
              P H H H             
              P P P               
                                  
                                  
                                  
                                  
                                  
                                  
                                  
Grid  273 :
                                  
                                  
                                  
                                  
                                  
                                  
                                  
              P P H               
              P H H            

                                  
                                  
                                  
                                  
                                  
Grid  290 :
                                  
                                  
                                  
                                  
                                  
                                  
                P P               
                P P               
                H P               
                H P               
                H                 
                                  
                                  
                                  
                                  
                                  
                                  
Grid  291 :
                                  
                                  
                                  
                                  
                                  
                               

In [358]:
# count = 0
# for i in direction_matrix:
#     if(len(i) < 8):
#         print(i, count)
#     count += 1

# print(direction_matrix[44])

In [357]:
# len(direction_matrix)

In [356]:
# len(best_grids)

In [322]:
#Filters for length of directions
count = 0
new_direction_matrix, new_score_matrix, new_best_grid = [], [], []
for i in direction_matrix:
    if(len(i) == 8):
        count_counter.append(count)
        new_direction_matrix.append(direction_matrix[count])
        new_score_matrix.append(score_matrix[count])
        new_best_grid.append(best_grids[count])

    count += 1

259


In [323]:
import random
top_100_direction_matrix, top_100_new_score_matrix, top_100_new_best_grid = zip(*random.sample(list(zip(new_direction_matrix, new_score_matrix, new_best_grid)), 100))

In [355]:
#  protein = 'HPPPPPPHH'
# for grid in range(len(top_100_new_best_grid)):

#         # Print the index of the grid
#         print("Grid ", grid, ":")

#         # For each row in the grid
#         for row in range(len(protein) * 2 - 1):

#             # For each column in the grid
#             for col in range(len(protein) * 2 - 1):

#                 # Print the character at the given row and
#                 # column
#                 print(top_100_new_best_grid[grid][row][col], end=' ')

#             # Print a line in between the rows
#             print('')

In [354]:
# for i in top_100_direction_matrix[0:10]:
#         print(i)

In [326]:
import numpy as np
import pandas as pd

In [352]:
def mutate(crossover_cutoff, protein):
    mutate_matrix = []
    count = 0
    while (len(mutate_matrix) < 100):
        rnd_Numb = randint(0,
                           len(top_100_direction_matrix) - 1)
        rnd_Numb2 = randint(0,
                            len(top_100_direction_matrix) - 1)
        combined = top_100_direction_matrix[
            rnd_Numb][:crossover_cutoff] + top_100_direction_matrix[rnd_Numb2][
                crossover_cutoff:]
        check = checkLegal(combined, protein)
        if (check == True):

            mutate_matrix.append(
                top_100_direction_matrix[rnd_Numb][:crossover_cutoff] +
                top_100_direction_matrix[rnd_Numb2][crossover_cutoff:])

    return mutate_matrix


# protein = 'HPPPPPPHH'
# mut = mutate(5, protein)
# len(mut)
# # mut

100

In [353]:
def getDirection(direction, orientation):
    rel_direction = ''
    if (orientation == -1):
        if(direction == 'F'):

            rel_direction = 'L'
            orientation = -1

        elif(direction == 'L'):

            rel_direction = 'D'
            orientation = -2

        elif(direction == 'R'):

            rel_direction = 'U'
            orientation = 2

    elif (orientation == 1):
        if(direction == 'F'):

            rel_direction = 'R'
            orientation = 1

        elif(direction == 'L'):

            rel_direction = 'U'
            orientation = 2

        elif(direction == 'R'):

            rel_direction = 'D'
            orientation = -2

    elif (orientation == 2):
        if(direction == 'F'):

            rel_direction = 'U'
            orientation = 2

        elif(direction == 'L'):
            rel_direction = 'L'
            orientation = -1

        elif(direction == 'R'):
            rel_direction = 'R'
            orientation = 1

    elif (orientation == -2):
        if(direction == 'F'):

            rel_direction = 'D'
            orientation = -2

        elif(direction == 'L'):

            rel_direction = 'R'
            orientation = 1

        elif(direction == 'R'):

            rel_direction = 'L'
            orientation = -1

    return rel_direction, orientation


def create_empty_grid(protein):
    #-Fill the grid with empty characters-#
    # For each row of the grid
    current_grid = []
    for row in range(len(protein) * 2 - 1):

        # Create a new empty list for that row
        current_grid.append([])

        # For each column of the grid
        for col in range(len(protein) * 2 - 1):

            # Add an empty character at the given row/column
            current_grid[row].append(EMPTY)

    # Start the first protein element in the middle of the grid
    current_row = current_col = int((len(protein) * 2 - 1) / 2)
    return current_grid


def checkLegal(mut_list, protein):
    grid = create_empty_grid(protein)
    orientation = 2
    current_row = current_col = int(len(grid)/2)

    count = 0
    for direction in mut_list:

        direction, orientation = getDirection(direction, orientation)
        if direction == 'R':
            current_col += 1
        elif direction == 'D':
            current_row += 1
        elif direction == 'L':
            current_col -= 1
        elif direction == 'U':
            current_row -= 1
        if(grid[current_row][current_col] != EMPTY):
            return False
        else:
            grid[current_row][current_col] = protein[count]
        count += 1
    return True